In [1]:
!pip install -q cassio datasets langchain openai tiktoken

In [2]:
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings

from datasets import load_dataset

import cassio

c:\Users\prasa\OneDrive\Documents\GitHub\Langchain\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
!pip install PyPDF2

     ---------------------------------------- 0.0/232.6 kB ? eta -:--:--
     -------------------------------------- 232.6/232.6 kB 7.2 MB/s eta 0:00:00


In [4]:
from PyPDF2 import PdfReader

In [5]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
ASTRA_DB_APPLICATION_TOKEN='AstraCS:OuLhfZbkLWmTLTnxPzIDOAkG:fb7adc432af2674036b725b6455d8c8bebe4be351acff913cd385377c302a34f'
ASTRA_DB_ID='a7631951-f191-466e-9c5f-b51cbe6885ae'

In [8]:
pdfreader=PdfReader('2009 Introduction to Algorithms Third Ed.pdf')

In [9]:
from typing_extensions import  Concatenate
raw_text=''
for i,page in enumerate(pdfreader.pages):
    content=page.extract_text()
    if content:
        raw_text+=content

In [10]:
cassio.init(token=os.environ['ASTRA_DB_APPLICATION_TOKEN'],database_id=os.environ['ASTRA_DB_ID'])

In [11]:
llm=OpenAI(openai_api_key=os.environ['OPENAI_API_KEY'])
embedding=OpenAIEmbeddings(openai_api_key=os.environ['OPENAI_API_KEY'])

In [12]:
astra_vector_store = Cassandra(embedding=embedding,
                               table_name='pdf_query_demo',
                               session=None,
                               keyspace=None)

In [13]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter=CharacterTextSplitter(
    separator='\n',
    chunk_size=800,
    chunk_overlap=200,
    length_function=len,
)
texts=text_splitter.split_text(raw_text)

Created a chunk of size 841, which is longer than the specified 800
Created a chunk of size 801, which is longer than the specified 800
Created a chunk of size 929, which is longer than the specified 800
Created a chunk of size 935, which is longer than the specified 800
Created a chunk of size 888, which is longer than the specified 800


In [16]:
astra_vector_store.add_texts(texts)
astra_vector_index=VectorStoreIndexWrapper(vectorstore=astra_vector_store)

In [18]:
print(astra_vector_index.query('Explain the median of medians algorithm',llm=llm))

 The median of medians algorithm is a divide and conquer approach to finding the median of a set of elements. It is an improvement on the basic divide and conquer algorithm, which has a worst-case running time of O(nlogn). The median of medians algorithm has a worst-case running time of O(n), making it more efficient for finding the median in certain cases.

The algorithm works by dividing the set of elements into groups of five and finding the median of each group. Then, it finds the median of these medians, called the "median-of-medians". This median-of-medians is used as the pivot for partitioning the elements into two subarrays. One subarray contains elements greater than the pivot, and the other contains elements less than the pivot. This process is repeated recursively until the median is found.

The key insight of the algorithm is that the median-of-medians will always be close to the true median, as it is guaranteed to be greater than at least half of the elements and less than